In [31]:
import importlib
from typing import Tuple
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import json
import sys
import os

sys.path.insert(1, '/home/maciek/Documents/Magisterka/kubernetes-anomaly-detector')
print(sys.path)


import kad.visualization.visualization as visualization
import kad.kad_utils.kad_utils as kad_utils
import kad.models_evaluation.models_evaluator as models_evaluator
import kad.model_selector.model_selector as model_selector
from kad.model import i_model, autoencoder_model, hmm_model, lstm_model
from kad.model import sarima_model
from sklearn.model_selection import train_test_split

models_evaluator = importlib.reload(models_evaluator)

kad_utils.customize_matplotlib()

In [32]:
SARIMA_KEY: str = "SARIMA"
AUTOENCODER_KEY: str = "autoencoder"
HMM_KEY: str = "HMM"
LSTM_KEY: str = "LSTM"

ACC_KEY: str = "accuracy"
VALID_ERR_KEY: str = "valid_err"
PREC_KEY: str = "precision"
RECALL_KEY: str = "recall"
AUROC_KEY: str = "auroc"
CUSTOM_KEY: str = "custom"

In [33]:

def downsampling_func(x: pd.DataFrame):
    """
    Values column is replaced with a mean, and labels are replaced by True if any of the labels
    in the original df was True
    :param x: part of dataframe to squash to one row
    :return: squashed pandas Series
    """
    values = x["value"].mean()

    labels = (x[kad_utils.GROUND_TRUTH_COLUMN]).any()

    return pd.Series([values, labels], index=["value", kad_utils.GROUND_TRUTH_COLUMN])

In [34]:
def perform_classical_evaluation(valid_err: float, evaluation_df: pd.DataFrame) -> dict:
    visualization.visualize(evaluation_df, metric_name="value")

    evaluator = models_evaluator.ModelsEvaluator(df=evaluation_df)

    print("Valid err: ", valid_err)
    print("Accuracy: ", evaluator.get_accuracy())
    print("Avg precision: ", evaluator.get_average_precision())
    print("Recall: ", evaluator.get_recall_score())
    print("AU ROC: ", evaluator.get_auroc())

    return {ACC_KEY: evaluator.get_accuracy(),
            # VALID_ERR_KEY: valid_err,
            PREC_KEY: evaluator.get_average_precision(),
            RECALL_KEY: evaluator.get_recall_score(),
            AUROC_KEY: evaluator.get_auroc()}


In [35]:
def perform_customized_evaluation(valid_err: float, evaluation_df: pd.DataFrame) -> dict:
    visualization.visualize(evaluation_df, metric_name="value")

    evaluator = models_evaluator.ModelsEvaluator(df=evaluation_df)

    print("Customized score: ", evaluator.get_customized_score())

    return {CUSTOM_KEY: evaluator.get_customized_score()}


In [36]:
def evaluate_sarima(p_preprocessed_df: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
    values_df = p_preprocessed_df[["value"]]
    tsa = model_selector.ModelSelector(values_df)
    dominant_freq = tsa.calculate_dominant_frequency()

    model: i_model.IModel = sarima_model.SarimaModel(order=(0, 0, 0), seasonal_order=(1, 0, 1, dominant_freq))

    train_df, test_df = train_test_split(values_df, shuffle=False, train_size=0.15)
    valid_err = model.train(train_df)

    results_df = pd.DataFrame()
    samples_to_predict = 18
    for i in range(0, len(test_df), samples_to_predict):
        results_df = model.test(test_df.iloc[i:i + samples_to_predict])

    results_df[kad_utils.GROUND_TRUTH_COLUMN] = p_preprocessed_df[kad_utils.GROUND_TRUTH_COLUMN]
    return valid_err, results_df.loc[test_df.index & results_df.index]

In [37]:
def evaluate_autoencoder(p_preprocessed_df: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
    model: i_model.IModel = autoencoder_model.AutoEncoderModel(time_steps=12)

    values_df = p_preprocessed_df[["value"]]
    train_df, test_df = train_test_split(values_df, shuffle=False, train_size=0.15)
    valid_err = model.train(train_df)

    results_df = pd.DataFrame()
    samples_to_predict = 60
    for i in range(0, len(test_df), samples_to_predict):
        if len(test_df.iloc[i:i + samples_to_predict]) < samples_to_predict:
            break
        results_df = model.test(test_df.iloc[i:i + samples_to_predict])

    results_df[kad_utils.GROUND_TRUTH_COLUMN] = p_preprocessed_df[kad_utils.GROUND_TRUTH_COLUMN]
    return valid_err, results_df.loc[test_df.index & results_df.index]

In [38]:
def evaluate_hmm(p_preprocessed_df: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
    model: i_model.IModel = hmm_model.HmmModel()

    values_df = p_preprocessed_df[["value"]]
    train_df, test_df = train_test_split(values_df, shuffle=False, train_size=0.15)
    valid_err = model.train(train_df)

    results_df = pd.DataFrame()
    samples_to_predict = 5
    for i in range(0, len(test_df), samples_to_predict):
        if len(test_df.iloc[i:i + samples_to_predict]) < samples_to_predict:
            break
        results_df = model.test(test_df.iloc[i:i + samples_to_predict])

    results_df[kad_utils.GROUND_TRUTH_COLUMN] = p_preprocessed_df[kad_utils.GROUND_TRUTH_COLUMN]
    return valid_err, results_df.loc[test_df.index & results_df.index]

In [39]:
def evaluate_lstm(p_preprocessed_df: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
    model: i_model.IModel = lstm_model.LstmModel(time_steps=12)

    values_df = p_preprocessed_df[["value"]]
    train_df, test_df = train_test_split(values_df, shuffle=False, train_size=0.15)
    valid_err = model.train(train_df)

    results_df = pd.DataFrame()
    samples_to_predict = 25
    for i in range(0, len(test_df), samples_to_predict):
        if len(test_df.iloc[i:i + samples_to_predict]) < samples_to_predict:
            break
        results_df = model.test(test_df.iloc[i:i + samples_to_predict])

    results_df[kad_utils.GROUND_TRUTH_COLUMN] = p_preprocessed_df[kad_utils.GROUND_TRUTH_COLUMN]
    return valid_err, results_df.loc[test_df.index & results_df.index]

In [40]:
from typing import Tuple
import concurrent

eval_dict: dict = {}

X_LABEL = "timestamp"
data_dir = "data/archive/"
file_dir = "artificialWithAnomaly"

with open("data/archive/combined_windows.json") as f:
    ground_true_anomalies = json.load(f)

executor = concurrent.futures.ProcessPoolExecutor()


def task_executor(filename: str, original_df: pd.DataFrame, ground_true_anomalies: dict, eval_func):
    file_eval_dict = {}

    true_anomalies_ranges = ground_true_anomalies["/".join(file_path.split("/")[-2:])]
    ground_true_anomalies_df = pd.DataFrame()
    for anom_range in true_anomalies_ranges:
        ground_true_anomalies_df = ground_true_anomalies_df.append(
            original_df.loc[anom_range[0]:anom_range[1]])
    original_df[kad_utils.GROUND_TRUTH_COLUMN] = [idx in ground_true_anomalies_df.index for idx in
                                                  original_df.index.tolist()]

    # SARIMA
    preprocessed_df = original_df.groupby(pd.Grouper(freq="h")).apply(downsampling_func)
    scaler = MinMaxScaler(feature_range=(0, 1))
    preprocessed_df["value"] = scaler.fit_transform(preprocessed_df.values)

    sarima_valid_err, sarima_eval_df = evaluate_sarima(preprocessed_df)
    file_eval_dict[SARIMA_KEY] = eval_func(sarima_valid_err, sarima_eval_df)

    # AUTOENCODER
    preprocessed_df = original_df.copy()
    preprocessed_df["value"] = preprocessed_df["value"].rolling("h").sum()
    scaler = MinMaxScaler(feature_range=(0, 1))
    preprocessed_df["value"] = scaler.fit_transform(preprocessed_df.values)

    autoenc_valid_err, autoenc_eval_df = evaluate_autoencoder(preprocessed_df)
    file_eval_dict[AUTOENCODER_KEY] = eval_func(autoenc_valid_err, autoenc_eval_df)

    # HMM
    preprocessed_df = original_df.groupby(pd.Grouper(freq="h")).apply(downsampling_func)
    scaler = MinMaxScaler(feature_range=(0, 1))
    preprocessed_df["value"] = scaler.fit_transform(preprocessed_df.values)

    hmm_valid_err, hmm_eval_df = evaluate_hmm(preprocessed_df)
    file_eval_dict[HMM_KEY] = eval_func(hmm_valid_err, hmm_eval_df)

    # LSTM
    preprocessed_df = original_df.groupby(pd.Grouper(freq="h")).apply(downsampling_func)
    scaler = MinMaxScaler(feature_range=(0, 1))
    preprocessed_df["value"] = scaler.fit_transform(preprocessed_df.values)

    lstm_valid_err, lstm_eval_df = evaluate_lstm(preprocessed_df)
    file_eval_dict[LSTM_KEY] = eval_func(lstm_valid_err, lstm_eval_df)

    return filename, file_eval_dict


files_to_eval = ["art_daily_flatmiddle.csv",
                 "art_daily_jumpsdown.csv",
                 "art_daily_nojump.csv",
                 "art_daily_jumpsup.csv"]

futures_table = list()

for (dirpath, dirnames, filenames) in os.walk(data_dir):
    for filename in filenames:
        if filename in files_to_eval:
            file_path = os.path.join(dirpath, filename)
            original_df = pd.read_csv(file_path, parse_dates=True, index_col="timestamp")
            print("after reading csv")
            futures_table.append(
                executor.submit(task_executor, filename, original_df, ground_true_anomalies, perform_customized_evaluation))

for future in futures_table:
    result: Tuple[str, dict] = future.result()
    eval_dict[result[0]] = result[1]

In [41]:
eval_json = "eval.json"

with open(eval_json, "w") as fp:
    json.dump(eval_dict, fp)
